In [ ]:
from sqlalchemy import create_engine, text
import pandas as pd
import boto3
import awswrangler as wr
import warnings
warnings.filterwarnings('ignore')

# ── Credentials ───────────────────────────────────────────
HOST     = 'AWS database Link'
PORT     = 3306
USER     = 'USERNAME'
PASSWORD = 'PASSWORD'

# STEP 1: Create the database (connect WITHOUT db name)
engine_root = create_engine(
    f"mysql+pymysql://{USER}:{PASSWORD}@{HOST}:{PORT}/",
    isolation_level = "AUTOCOMMIT" 
)

with engine_root.connect() as conn:
    conn.execute(text("CREATE DATABASE IF NOT EXISTS vendor_sales;"))
    print(" Database 'vendor_sales' created!")

engine_root.dispose()

# STEP 2: Now connect WITH the database name
engine = create_engine(
    f"mysql+pymysql://{USER}:{PASSWORD}@{HOST}:{PORT}/vendor_sales"  # ← vendor_sales here
)

# Test connection
with engine.connect() as conn:
    conn.execute(text("SELECT 1;"))
    print(" Connected to vendor_sales database!")

# STEP 3: Load files from S3

boto3.setup_default_session(
    aws_access_key_id     = 'S3 BUCKET KEY ID',
    aws_secret_access_key = 'fAWS BUCKET SECRET ACCESS KEY',
    region_name           = 'ap-southeast-1'
)

bucket = "s3://vendor-sales-raw/raw/"

print("\n Loading files from S3...")
sales           = wr.s3.read_csv(f"{bucket}sales.csv")
purchases       = wr.s3.read_csv(f"{bucket}purchases.csv")
vendor_invoice  = wr.s3.read_csv(f"{bucket}vendor_invoice.csv")
purchase_prices = wr.s3.read_csv(f"{bucket}purchase_prices.csv")
begin_inventory = wr.s3.read_csv(f"{bucket}begin_inventory.csv")
end_inventory   = wr.s3.read_csv(f"{bucket}end_inventory.csv")
print(" All 6 files loaded from S3!")

# STEP 4: Upload all 6 tables to RDS
datasets = {
    'sales'          : sales,
    'purchases'      : purchases,
    'vendor_invoice' : vendor_invoice,
    'purchase_prices': purchase_prices,
    'begin_inventory': begin_inventory,
    'end_inventory'  : end_inventory
}

print("\n Uploading tables to RDS MySQL...")
for table_name, df in datasets.items():
    df.to_sql(
        name      = table_name,
        con       = engine,
        if_exists = 'replace',
        index     = False,
        chunksize = 1000
    )
    print(f"   {table_name:20s} → {df.shape[0]:>6} rows, {df.shape[1]} cols")

print("\n All 6 tables uploaded to RDS!")


# STEP 5: Verify all tables exist
with engine.connect() as conn:
    result = conn.execute(text("SHOW TABLES;"))
    print("\n📋 Tables in vendor_sales:")
    for row in result:
        print(f"   → {row[0]}")

engine.dispose()
print("\n Connection closed.")

✅ Database 'vendor_sales' created!
✅ Connected to vendor_sales database!

📥 Loading files from S3...
✅ All 6 files loaded from S3!

📤 Uploading tables to RDS MySQL...
  ✅ sales                → 12825363 rows, 14 cols
  ✅ purchases            → 2372474 rows, 16 cols
  ✅ vendor_invoice       →   5543 rows, 10 cols
  ✅ purchase_prices      →  12261 rows, 9 cols
  ✅ begin_inventory      → 206529 rows, 9 cols
  ✅ end_inventory        → 224489 rows, 9 cols

🎉 All 6 tables uploaded to RDS!

📋 Tables in vendor_sales:
   → begin_inventory
   → end_inventory
   → purchase_prices
   → purchases
   → sales
   → vendor_invoice

🔒 Connection closed.
